In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
from dateutil import parser
from datetime import datetime

In [2]:
# Constants
chrome_path = "C:/Users/janin/OneDrive/Documents/GitHub/chromedriver.exe"
mars_sites = [{"Name":"NASA Mars Explorer News",
               "URL":"https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest",
               "Type":"News",
               "Link Stem":"https://mars.nasa.gov"
              },
              {"Name":"JPL Mars Images",
               "URL":"https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"},
              {"Name":"Mars Weather",
               "URL":"https://twitter.com/marswxreport?lang=en"},
              {"Name":"Mars Facts",
               "URL":"https://space-facts.com/mars/"},
              {"Name":"Mars Hemispheres",
               "URL":"https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"}]

In [3]:
def init_browser():
    executable_path = {"executable_path": chrome_path}
    return Browser("chrome", **executable_path, headless=False)

In [4]:
def get_page(url):
    browser = init_browser()
    browser.visit(url)
    html = browser.html
    return BeautifulSoup(html, "html.parser")

In [10]:
def scrape_news(site):
    print("News Article")
    print("____________")
    soup = get_page(site['URL'])
    article = {}
    articles = soup.find_all("div",class_ = "list_text")
    latest_date = datetime(2000, 1, 1, 0, 0)
    article_url = ""
    article_title = ""
    description = ""
    for article in articles:
        date_text = article.find("div",class_="list_date").text
        article_date = datetime.strptime(date_text,'%B %d, %Y')
        if article_date > latest_date:
            latest_date = article_date
            print(article_date)
            article_link = article.find("div",class_="content_title")
            article_url = f"{site['Link Stem']}{article_link.a['href']}"
            print(article_url)
            article_title = article_link.a.text.replace('\n','').strip()
            print(article_title)
            description = article.find("div", class_ = "article_teaser_body")
            article_description = description.text.strip()
            print(article_description)
    print("____________")
    article["Title"] = article_title
    article["Link URL"] = article_url
    article["Description"] = description
    return article
            

In [11]:
def get_post(site):
    post = {}
    post.update({"Site":site["Name"]})
    url = site["URL"]
    post["Site URL"] = url
    site_type = site["Type"]
    post["Site Type"] = site_type
    if site['Type'] == "News":
        details = scrape_news(site)
        post["Detail"] = details["Title"]
        post["Detail URL"] = details["Link URL"]
        post["Detail Description"] = details["Description"]
    return post

In [12]:
site = mars_sites[0]
post = get_post(site)
print(post)

News Article
____________
2019-11-15 00:00:00
https://mars.nasa.gov/news/8551/mars-scientists-investigate-ancient-life-in-australia/
Mars Scientists Investigate Ancient Life in Australia
Teams with NASA's Mars 2020 and ESA's ExoMars practiced hunting for fossilized microbial life in the Australian Outback in preparation for their Red Planet missions.
____________
{'Site': 'NASA Mars Explorer News', 'Site URL': 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest', 'Site Type': 'News', 'Detail': 'Mars Scientists Investigate Ancient Life in Australia', 'Detail URL': 'https://mars.nasa.gov/news/8551/mars-scientists-investigate-ancient-life-in-australia/', 'Detail Description': <div class="article_teaser_body">Teams with NASA's Mars 2020 and ESA's ExoMars practiced hunting for fossilized microbial life in the Australian Outback in preparation for their Red Planet missions. </div>}


In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.mars_db
collection = db.items
for site in mars_sites:
    response = requests.get(site["URL"])
    soup = BeautifulSoup(response.text, 'lxml')
    results = soup.find_all('li', class_=site["Class"])
    for result in results:
        